## Script to clip the index raster to each NHM HRU

Theodore Barnhart | tbarnhart@usgs.gov

In [1]:
import geopandas as gpd
import subprocess

In [2]:
files = glob.glob('./data/nhrus/clean_AEA/*.shp')

In [16]:
def make_outpath(df):
    outpath = './data/nhrus/AEA_tiffs/HUC_%s_nhruID_%s.tiff'%(df.region,df.nhruID)
    return outpath

In [21]:
def clip_raster(df):
    
    cutline = df.cutline
    feature = df.nhruID
    inpath = df.inpath
    outpath = df.outpath
    
    cmd = "gdalwarp -cutline %s -cwhere hru_id_nat=%s -crop_to_cutline -overwrite %s %s"%(cutline,
        feature,inpath,outpath)
    
    subprocess.call(cmd,shell=True)
    
    return None

In [22]:
def runClip(fl,inpath='tmp'):
    tmp = gpd.read_file(fl)
    dat = pd.DataFrame({'nhruID':tmp.hru_id_nat.unique()}) #dataframe of the unique NHM HRU identifiers
    del tmp # close tmp file
    
    reg = fl.split('_')[-2] # extract the region
    
    dat['cutline'] = fl # insert the shapefile as the cutline
    dat['region'] = reg
    dat['inpath'] = inpath # specified
    dat['outpath'] = dat.apply(make_outpath,axis=1)
    
    dat.apply(clip_raster,axis=1) # run the clip code
    
    # test that the number of files created is the same as the number of features
    files = glob.glob('./data/nhrus/AEA_tiffs/HUC_%s_*.tiff'%(reg))
    if (len(files)-len(dat)) != 0:
        return 'Error!'

In [23]:
runClip(files[0],inpath='./data/hrap_grid_AEA_sm.tiff')

KeyboardInterrupt: 